In [10]:
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import mne
from mne.stats import summarize_clusters_stc

import scipy
import pickle

from mayavi import mlab
from IPython.display import Image


In [11]:
tmin, tmax = -0.1, 0.6

source_MainFolder = "..\SourceLocalization\SourceEstimates\Data="
sourceFolder = source_MainFolder + str(tmin) + '_' + str(tmax) + '\\'
print('Source folder: ', sourceFolder)

statResultsFolder  = '..\SourceLocalization\Results\Data='
statResultsFolder = statResultsFolder + str(tmin) + '_' + str(tmax) + '\\'
print('Statisctics results folder: ', statResultsFolder)    


subjects_dir = '..\SourceLocalization\subjects\\'
print('Subjects directory: ', subjects_dir)

tstep= 0.01
inv_sol_method = 'beamformer'
smoothAmount=70

Source folder:  ..\SourceLocalization\SourceEstimates\Data=-0.1_0.6\
Statisctics results folder:  ..\SourceLocalization\Results\Data=-0.1_0.6\
Subjects directory:  ..\SourceLocalization\subjects\


In [12]:
# generate the inverse solution for group average
def prepareInverseSolution_group(data, tstep, tmin_tmp=0):
    
    src_ave = mne.read_source_spaces(subjects_dir+'fsaverage/bem/fsaverage-ico-5-src.fif')

    fsave_vertices = [s['vertno'] for s in src_ave]        
            
    stc_return = mne.SourceEstimate(data, fsave_vertices, tmin_tmp, tstep, subject='fsaverage')
    
    
    return stc_return
    

Visualize source estimates

In [26]:
mlab.init_notebook('png')
def showResult(s_id, sourceFolder, stc, condName, minimum, maximum, tmin_tmp=0):
    
    initial_time = tmin_tmp
    hemi_list = ['rh', 'lh']
    for hemi in hemi_list:
        print('Hemi: ', hemi)
        #colors = np.array(([255,255,255], [250,128,114], [255,36,0], [124,10,2]))/256
        kwargs = dict(initial_time=initial_time, surface='inflated', hemi=hemi, subjects_dir=subjects_dir,
                      verbose=True, size=(600, 600), spacing='all') #, clim=dict(kind='value', pos_lims=(10, 100, 400))
        
        brain = stc.plot(**kwargs, colormap='Oranges')
        brain.scale_data_colormap(fmin=minimum, fmid=(maximum+minimum)/2, fmax=maximum, transparent=True)
        
        

        #brain.set_data_time_index(time_idx)
        brain.show_view('lateral');
        #mlab.show()

        brain.save_image(sourceFolder + s_id + '_' + hemi + '_' + condName + '_vrtx.png')

        Image(filename = sourceFolder +  s_id + '_' + hemi + '_' + condName + '_vrtx.png', width=600)


Notebook initialized with png backend.


In [14]:
tminData_cls_tmp, tmaxData_cls_tmp = 0.1, 0.4

stc_real_filename = 'stc_fsave_allTogether_real_onActivationMaps_'+inv_sol_method+'_'+str(tminData_cls_tmp)+'_'+str(tmaxData_cls_tmp)+'_sm='+str(smoothAmount)+'.npy'
print(statResultsFolder+stc_real_filename)

stc_fsave_all_real_clfRange = np.load(statResultsFolder+stc_real_filename)
print('shape of real sounds source estimates: ', stc_fsave_all_real_clfRange.shape)



..\SourceLocalization\Results\Data=-0.1_0.6\stc_fsave_allTogether_real_onActivationMaps_beamformer_0.1_0.4_sm=70.npy
shape of real sounds source estimates:  (20484, 31, 21)


In [15]:
tminData_baseline_tmp, tmaxData_baseline_tmp = -0.04, 0

stc_real_filename_baseline = 'stc_fsave_allTogether_real_onActivationMaps_'+inv_sol_method+'_'+str(tminData_baseline_tmp)+'_'+str(tmaxData_baseline_tmp)+'_sm='+str(smoothAmount)+'.npy'
print(stc_real_filename_baseline)
stc_fsave_all_real_baseline = np.load(statResultsFolder+stc_real_filename_baseline)
print('shape of real sounds baseline source estimates: ', stc_fsave_all_real_baseline.shape)

stc_fsave_allTogether_real_onActivationMaps_beamformer_-0.04_0_sm=70.npy
shape of real sounds baseline source estimates:  (20484, 5, 21)


In [16]:
# take mean over participants (dimension 2)
stc_fsave_all_real_clfRange_avg = np.mean(stc_fsave_all_real_clfRange, axis=2)
print('Shape stc_fsave_all_real_clfRange_avg after avg over subjects: ', stc_fsave_all_real_clfRange_avg.shape)
# take mean over time points (dimension 1)
stc_fsave_all_real_clfRange_avg= np.mean(stc_fsave_all_real_clfRange_avg, axis=1)
print('Shape stc_fsave_all_real_clfRange_avg after avg across time: ', stc_fsave_all_real_clfRange_avg.shape)
stc_fsave_all_real_clfRange_avg.shape

# take mean over participants (dimension 2)
stc_fsave_all_real_baseline_avg = np.mean(stc_fsave_all_real_baseline, axis=2)
print('Shape stc_fsave_all_real_baseline_avg after avg over subjects: ', stc_fsave_all_real_baseline_avg.shape)
# take mean over time points (dimension 1)
stc_fsave_all_real_baseline_avg = np.mean(stc_fsave_all_real_baseline_avg, axis=1)
print('Shape stc_fsave_all_real_baseline_avg after avg across time: ', stc_fsave_all_real_baseline_avg.shape)
stc_fsave_all_real_baseline_avg.shape

Shape stc_fsave_all_real_clfRange_avg after avg over subjects:  (20484, 31)
Shape stc_fsave_all_real_clfRange_avg after avg across time:  (20484,)
Shape stc_fsave_all_real_baseline_avg after avg over subjects:  (20484, 5)
Shape stc_fsave_all_real_baseline_avg after avg across time:  (20484,)


(20484,)

### Plot single subject sources

In [17]:
# Take difference post-base normalized:

stc_fsave_all_real_diff_avg = 100*(stc_fsave_all_real_clfRange_avg - stc_fsave_all_real_baseline_avg)/stc_fsave_all_real_baseline_avg



In [18]:
stc_diff = prepareInverseSolution_group(stc_fsave_all_real_diff_avg, tstep=tstep, tmin_tmp=tminData_cls_tmp)

    Reading a source space...
    [done]
    Reading a source space...
    [done]
    2 source spaces read


In [19]:
print('Max val: ', np.max(stc_diff.data))


Max val:  1409.9375374371173


In [101]:
print('Max val: ', np.max(stc_diff.data))

v_ma = 1100
v_mi = 500

showResult('fsaverage', sourceFolder, stc_diff, 
           'real_change_allTogether_norm2z_tp10'+inv_sol_method+'_sm='+str(smoothAmount), 
           minimum=v_mi, maximum=v_ma)#, med = (v_mi + v_ma)/2)

Max val:  1409.9375374371173
Hemi:  rh
Using control points [ 822.19922909  868.37325414 1147.4512822 ]
Hemi:  lh
Using control points [ 822.19922909  868.37325414 1147.4512822 ]


Get labels of parcellation

In [102]:
#sd = '/Users/athina/Documents/Projects/Pinar_MEG/'

src_ave = mne.read_source_spaces(subjects_dir+'fsaverage\\bem\\fsaverage-ico-5-src.fif')

fsave_vertices = [s['vertno'] for s in src_ave]

label = mne.read_labels_from_annot('fsaverage', parc='aparc',hemi = 'both',
                                   subjects_dir=subjects_dir)
print('All labels: ')
label

    Reading a source space...
    [done]
    Reading a source space...
    [done]
    2 source spaces read
Reading labels from parcellation...
   read 35 labels from ..\SourceLocalization\subjects\fsaverage\label\lh.aparc.annot
   read 34 labels from ..\SourceLocalization\subjects\fsaverage\label\rh.aparc.annot
All labels: 


[<Label | fsaverage, 'bankssts-lh', lh : 2137 vertices>,
 <Label | fsaverage, 'bankssts-rh', rh : 2196 vertices>,
 <Label | fsaverage, 'caudalanteriorcingulate-lh', lh : 1439 vertices>,
 <Label | fsaverage, 'caudalanteriorcingulate-rh', rh : 1608 vertices>,
 <Label | fsaverage, 'caudalmiddlefrontal-lh', lh : 3736 vertices>,
 <Label | fsaverage, 'caudalmiddlefrontal-rh', rh : 3494 vertices>,
 <Label | fsaverage, 'cuneus-lh', lh : 1630 vertices>,
 <Label | fsaverage, 'cuneus-rh', rh : 1638 vertices>,
 <Label | fsaverage, 'entorhinal-lh', lh : 1102 vertices>,
 <Label | fsaverage, 'entorhinal-rh', rh : 902 vertices>,
 <Label | fsaverage, 'frontalpole-lh', lh : 272 vertices>,
 <Label | fsaverage, 'frontalpole-rh', rh : 369 vertices>,
 <Label | fsaverage, 'fusiform-lh', lh : 4714 vertices>,
 <Label | fsaverage, 'fusiform-rh', rh : 4661 vertices>,
 <Label | fsaverage, 'inferiorparietal-lh', lh : 7871 vertices>,
 <Label | fsaverage, 'inferiorparietal-rh', rh : 9676 vertices>,
 <Label | fsavera

In [103]:
# Initialize a list of labels:

labels_names = stc_diff.data.shape[0]*[None]
labels_inSourceEst = stc_diff.data.shape[0]*[None]

for l_i in range(len(label)):

#    l_i = 0 # label index we want

    # the name for current labels:
    for ver in label[l_i].restrict(src_ave).vertices:
        labels_names[ver] = label[l_i].restrict(src_ave).name
        labels_inSourceEst[ver] = label[l_i].restrict(src_ave)


In [104]:
print(len(stc_diff.data))
print(len(labels_names))

20484
20484


In [105]:
desired_thres = 1000

sign_i = np.where(stc_fsave_all_real_diff_avg > desired_thres)[0]
labels_names_sign = []
for s in sign_i:
    print(labels_names[s])
    labels_names_sign.append(labels_names[s])

superiorfrontal-lh
precuneus-lh
rostralmiddlefrontal-rh
superiorfrontal-lh
rostralanteriorcingulate-lh
supramarginal-rh
rostralmiddlefrontal-rh
superiorfrontal-lh
superiorfrontal-lh
supramarginal-rh
precentral-rh
rostralmiddlefrontal-rh
rostralmiddlefrontal-rh
superiorfrontal-lh
superiorfrontal-lh
rostralanteriorcingulate-lh
inferiorparietal-rh
rostralmiddlefrontal-rh
rostralmiddlefrontal-rh
superiorfrontal-lh
superiorfrontal-lh
superiorfrontal-lh
superiorfrontal-lh
superiorfrontal-lh
precuneus-lh
rostralmiddlefrontal-rh
rostralanteriorcingulate-lh
precentral-rh
rostralmiddlefrontal-rh
rostralmiddlefrontal-rh
superiorfrontal-lh
precuneus-lh
precuneus-lh
rostralmiddlefrontal-rh
postcentral-rh
superiorfrontal-lh
rostralmiddlefrontal-rh
rostralmiddlefrontal-rh
rostralmiddlefrontal-rh
superiorfrontal-lh
superiorfrontal-lh
superiorfrontal-lh
superiorfrontal-lh
superiorfrontal-lh
precuneus-lh
supramarginal-rh
precuneus-lh
precuneus-lh
superiorfrontal-lh
insula-rh
postcentral-rh
None
None
Non

In [106]:
len(sign_i)

106

In [107]:
labels_names_sign

['superiorfrontal-lh',
 'precuneus-lh',
 'rostralmiddlefrontal-rh',
 'superiorfrontal-lh',
 'rostralanteriorcingulate-lh',
 'supramarginal-rh',
 'rostralmiddlefrontal-rh',
 'superiorfrontal-lh',
 'superiorfrontal-lh',
 'supramarginal-rh',
 'precentral-rh',
 'rostralmiddlefrontal-rh',
 'rostralmiddlefrontal-rh',
 'superiorfrontal-lh',
 'superiorfrontal-lh',
 'rostralanteriorcingulate-lh',
 'inferiorparietal-rh',
 'rostralmiddlefrontal-rh',
 'rostralmiddlefrontal-rh',
 'superiorfrontal-lh',
 'superiorfrontal-lh',
 'superiorfrontal-lh',
 'superiorfrontal-lh',
 'superiorfrontal-lh',
 'precuneus-lh',
 'rostralmiddlefrontal-rh',
 'rostralanteriorcingulate-lh',
 'precentral-rh',
 'rostralmiddlefrontal-rh',
 'rostralmiddlefrontal-rh',
 'superiorfrontal-lh',
 'precuneus-lh',
 'precuneus-lh',
 'rostralmiddlefrontal-rh',
 'postcentral-rh',
 'superiorfrontal-lh',
 'rostralmiddlefrontal-rh',
 'rostralmiddlefrontal-rh',
 'rostralmiddlefrontal-rh',
 'superiorfrontal-lh',
 'superiorfrontal-lh',
 'supe

Brain = mne.viz.get_brain_class()

subjects_dir = mne.datasets.sample.data_path() + '/subjects'
#mne.datasets.fetch_hcp_mmp_parcellation(subjects_dir=subjects_dir,verbose=True)

#mne.datasets.fetch_aparc_sub_parcellation(subjects_dir=subjects_dir,verbose=True)
hemi = 'lh'
brain = Brain('fsaverage', hemi, 'inflated', subjects_dir=subjects_dir,
              cortex='low_contrast', background='white', size=(800, 600))
#brain.add_annotation('HCPMMP1')
labels_to_plot = [l for l in label if l.restrict(src_ave).name in labels_names_sign and hemi in l.name]
for l in labels_to_plot:
    brain.add_label(l, borders=True)

brain.show_view('lateral');
#Image(filename = sourceFolder + 'corticalSources.png', width=600)
brain.save_image(sourceFolder + 'corticalSources_'+hemi+'.png', 'rgb')

In [108]:
#brain.labels

Get MNI coordinates of the peak on left hemisphere

In [109]:
hemi = 'lh'
hemi_ind = 0 # lh
    
# find the peak on the lh
peak_vertex, peak_time = stc_diff.get_peak(hemi=hemi)
print('peak_vertex: ', peak_vertex)

# get the vertex at the peak
peak_vertex_surf = stc_diff.lh_vertno[peak_vertex]
import cmasher as cmr
cmap = plt.get_cmap('cmr.seasons')
   
# plot the source estimate with the peak painted blue
kwargs = dict(initial_time=0.1, surface='inflated', hemi=hemi, subjects_dir=subjects_dir,
                      verbose=True, size=(600, 600), spacing='all',  background='w', 
                      cortex=(211/256,211/256,211/256)) #, clim=dict(kind='value', pos_lims=(10, 100, 400))
        
brain = stc_diff.plot(**kwargs, colormap='Oranges')
brain.scale_data_colormap(fmin=v_mi, fmid=(v_mi+v_ma)/2, fmax=v_ma, transparent=True)

# add the vertex at the peak to the plot
#brain.add_foci(peak_vertex_surf, coords_as_verts=True, hemi=hemi, color='blue')

view = 'lateral' #medial lateral
brain.show_view(view);
brain.save_image(sourceFolder +  'all_' + hemi + '_with_maxpoint_'+view+'.png')

# convert vertex to MNI coordinates
coordinate = mne.vertex_to_mni(peak_vertex_surf, hemis=hemi_ind, subject='fsaverage', subjects_dir=subjects_dir)

print('coordinates: ', coordinate)

peak_vertex:  3416
Using control points [ 822.19922909  868.37325414 1147.4512822 ]


OSError: [Errno 22] Invalid argument: '..\\SourceLocalization\\SourceEstimates\\Data=-0.1_0.6\\all_lh_with_maxpoint_lateral.png'

Get MNI coordinates of the peak on right hemisphere

In [98]:
v_mi

600

In [100]:
hemi = 'rh'
hemi_ind = 1 #rh

    
# find the peak on the rh
peak_vertex, peak_time = stc_diff.get_peak(hemi=hemi)
print('peak_vertex: ', peak_vertex)

# get the vertex at the peak
peak_vertex_surf = stc_diff.rh_vertno[peak_vertex]

# plot the source estimate with the peak painted blue
kwargs = dict(initial_time=0.1, surface='inflated', hemi=hemi, subjects_dir=subjects_dir,
                      verbose=True, size=(600, 600), spacing='all', background='w', 
                      cortex=(211/256,211/256,211/256)) #, clim=dict(kind='value', pos_lims=(10, 100, 400))
        
brain = stc_diff.plot(**kwargs, colormap='Oranges')
brain.scale_data_colormap(fmin=v_mi, fmid=(v_mi+v_ma)/2, fmax=v_ma, transparent=True)

# add the vertex at the peak to the plot
#brain.add_foci(peak_vertex_surf, coords_as_verts=True, hemi=hemi, color='blue')

view = 'lateral' #medial lateral
brain.show_view(view);
brain.save_image(sourceFolder +  'all_' + hemi + '_with_maxpoint_'+view+'.png')

# convert vertex to MNI coordinates
coordinate = mne.vertex_to_mni(peak_vertex_surf, hemis=hemi_ind, subject='fsaverage', subjects_dir=subjects_dir)

print('coordinates: ', coordinate)




peak_vertex:  4514
Using control points [ 822.19922909  868.37325414 1147.4512822 ]
coordinates:  [  5.6532836  -12.23433971  53.08460617]


In [54]:
#labels_names_sign

In [55]:
#labels_rh_temporal

Get coordinates of the cluster peak on temporal lobe in right hemisphere

In [58]:
hemi = 'rh'
hemi_ind = 1

# find the labels of desired parcellations (e.g. temporal ones) 
labels_rh_temporal = [l for l in label if l.name in labels_names_sign and hemi in l.name][5]
print(labels_rh_temporal)

# restrict the area to find the peak in the preferred area
peak_temporal_rh, peak_time = stc_diff.in_label(labels_rh_temporal).get_peak()
print(peak_temporal_rh)

# get vertex at the peak
peak_vertex_surf_temporal_rh = stc_diff.rh_vertno[peak_temporal_rh]

# get MNI coordinates of the vertex at the peak
coordinate_temporal = mne.vertex_to_mni(stc_diff.rh_vertno[peak_vertex_surf_temporal_rh],
                                        hemis=hemi_ind, subject='fsaverage',
                                        subjects_dir=subjects_dir)

print('coordinates: ', coordinate_temporal)

<Label | fsaverage, 'supramarginal-rh', rh : 8150 vertices>
9791
coordinates:  [ 34.03600311 -30.93478394  16.78090858]


In [57]:
# plot source estimate with the peak painted blue
kwargs = dict(initial_time=0.1, surface='inflated', hemi=hemi, subjects_dir=subjects_dir,
                      verbose=True, size=(600, 600), spacing='all', background='w', 
                      cortex=(211/256,211/256,211/256))
        
brain = stc_diff.plot(**kwargs, colormap='Oranges')
#brain.add_foci(peak_vertex_surf_temporal_rh, coords_as_verts=True, hemi=hemi, color='blue')
brain.scale_data_colormap(fmin=v_mi, fmid=(v_mi+v_ma)/2, fmax=v_ma, transparent=True)

view = 'lateral' #medial
brain.show_view(view);
brain.save_image(sourceFolder +  'all_' + hemi + '_with_maxpoint_'+view+'_temporal.png')


Using control points [ 822.19922909  868.37325414 1147.4512822 ]
